In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000)

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics
import branca

import shared_utils
import utils

from siuba import *
import pandas as pd
import geopandas as gpd
import shapely

import datetime as dt
import time
from zoneinfo import ZoneInfo

import rt_analysis as rt
import importlib

import gcsfs
fs = gcsfs.GCSFileSystem()

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
E0414 22:32:29.616064981    1529 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0414 22:32:32.149091126    1529 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


In [2]:
importlib.reload(utils)
importlib.reload(rt)

<module 'rt_analysis' from '/home/jovyan/data-analyses/rt_delay/rt_analysis.py'>

In [3]:
air_joined = pd.read_parquet('airtable_joined.parquet')

In [4]:
ran_operators = []

In [5]:
pbar = tqdm()

0it [00:00, ?it/s]

#### Run all operators

In [6]:
fs_list = fs.ls(f'{utils.GCS_FILE_PATH}rt_trips/')

In [7]:
ran_operators = [int(path.split('rt_trips/')[1].split('_')[0])
                 for path in fs_list
                 if path.split('rt_trips/')[1]]

In [8]:
# ran_operators = []

In [9]:
# for agency in air_joined.calitp_itp_id.unique():
for agency in []:
    not_ran_operators = []
    if agency in ran_operators:
        print(f'already ran: {agency}')
        continue
    analysis_date = dt.date(2022, 3, 30) ##wednesday, new tables
    day = str(analysis_date.day).zfill(2)
    month = str(analysis_date.month).zfill(2)
    print(f'calculating for agency: {agency}...')
    try:
        rt_day = rt.OperatorDayAnalysis(agency, analysis_date, pbar)
        delay_to_parquet = rt_day.stop_delay_view.copy()
        delay_to_parquet['delay_seconds'] = delay_to_parquet.delay.map(lambda x: x.seconds)
        delay_to_parquet['arrival_time'] = delay_to_parquet.arrival_time.map(lambda x: x.isoformat())
        delay_to_parquet['actual_time'] = delay_to_parquet.actual_time.map(lambda x: x.isoformat())
        delay_to_parquet = delay_to_parquet >> select(-_.delay)
        shared_utils.utils.geoparquet_gcs_export(delay_to_parquet,
                                         f'{utils.GCS_FILE_PATH}stop_delay_views/',
                                        f'{agency}_{month}_{day}'
                                        )
        # rt_day.set_filter(start_time='15:00', end_time='19:00')
        # m = rt_day.segment_speed_map(how='low_speeds', size = [1300, 700])
        # shared_utils.utils.geoparquet_gcs_export(rt_day.stop_segment_speed_view,
        #                                          f'{utils.GCS_FILE_PATH}segment_speed_views/',
        #                                         f'{agency}_{month}_{day}_pm_peak'
        #                                         )
        rt_day.rt_trips.to_parquet(f'{utils.GCS_FILE_PATH}rt_trips/{agency}_{month}_{day}.parquet') ## early step towards scaling
        # m.save(f'./speedmaps/{agency}_02_{day}_pm_peak.html')
        # rt_day.stop_segment_speed_view.to_parquet(f'{utils.GCS_FILE_PATH}segment_speed_views/{agency}_02_{day}_pm_peak.parquet') ## early step towards scaling
        delay_to_parquet = rt_day.endpoint_delay_view.copy()
        delay_to_parquet['delay_seconds'] = delay_to_parquet.delay.map(lambda x: x.seconds)
        delay_to_parquet = delay_to_parquet >> select(-_.delay)
        shared_utils.utils.geoparquet_gcs_export(delay_to_parquet,
                                         f'{utils.GCS_FILE_PATH}endpoint_delay_views/',
                                        f'{agency}_{month}_{day}_pm_peak'
                                        )
        print(f'complete for agency: {agency}')
    except Exception as e:
        print(f'rt failed for agency {agency}')
        not_ran_operators += [agency]
        print(e)

In [10]:
import yaml

In [11]:
with open('../reports_config.yml') as reports_config:
    reports_config_data = yaml.load(reports_config, yaml.Loader)

In [12]:
reports_config_data

{'dla': {'notebook': './dla/e76_obligated_funds/prototype-district.ipynb',
  'params': {'district': [4, 7]}},
 'rt': {'notebook': './rt_delay/15_parameterize.ipynb',
  'params': {'itp_id': [300]}}}

In [13]:
reports_config_data['rt']['params']['itp_id'] = ran_operators

In [14]:
output = yaml.dump(reports_config_data)

In [15]:
# reports_config_data

In [16]:
with open('../reports_config.yml', 'w') as reports_config:
    reports_config.write(output)

### Ongoing issues
* 358 Union City too many trips without shape?

#### Parameters
* 159 1+ maps don't render
* 167 map_from_metrics fails

In [11]:
# air_joined['url'] = air_joined.apply(lambda x:
#                     f'http://docs.calitp.org/data-analyses/rt_delay/speedmaps/{x.calitp_itp_id}_02_08_pm_peak.html',
#                                     axis = 1)

# air_joined.to_csv('linked.csv')